In [4]:
%load_ext autoreload
%autoreload 2

import json
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def get_metrics(datasets, experiment_name, metrics_list):
    """Given a list of datasets, compute the average metrics for each dataset."""
    metrics = [[] for _ in metrics_list]
    for dataset in datasets:
        filename = f"/shared/ethanweber/magic-eraser/nerf-renders/{dataset}/{dataset}---{experiment_name}/{dataset}---{experiment_name}.json"
        with open(filename, "r") as f:
            metrics_dict = json.load(f)
            for idx, metric_name in enumerate(metrics_list):
                metrics[idx].append(metrics_dict[metric_name])
    # get the mean
    metrics = [sum(metric) / len(metric) for metric in metrics]
    return metrics

In [13]:
TEMPLATE = \
"""
\\begin{table*}[]
    \\centering
    \\begin{tabular}{l|lll|ll|lllll|l} % the number of columns needed
        \\toprule
        COLUMNS \\\\
        \\midrule
        EXPERIMENT_LINES
        \\bottomrule
    \\end{tabular}
    \\caption{\\textbf{CAPTION_TITLE.}}
    \\label{tab:TABLE_LABEL}
\\end{table*}
"""

In [10]:
sorted(['flowers', 'pipe', 'roses', 'pikachu', 'car', 'aloe', 'picnic', 'table', 'century', 'art', 'plant', 'garbage'])

['aloe',
 'art',
 'car',
 'century',
 'flowers',
 'garbage',
 'picnic',
 'pikachu',
 'pipe',
 'plant',
 'roses',
 'table']

In [14]:
COLUMNS = "\n & PSNR $\\uparrow$ & SSIM $\\uparrow$ & LPIPS $\\downarrow$ & Depth $\\downarrow$ & Disp. $\\downarrow$ & Mean $^{\\circ}$ $\\downarrow$ & Median $^{\\circ}$ $\\downarrow$ & \\% $30^{\\circ}$ $\\uparrow$ & Coverage $\\uparrow$"
METRICS_LIST = ["psnr", "ssim", "lpips", "depth", "disparity", "normals", "normals_median", "normals_30", "coverage"]
SIGFIGS = [2, 4, 4, 2, 2, 2, 2, 2, 2]
assert len(METRICS_LIST) == len(SIGFIGS)
# methods are the columns
# here we map the title in the column to the the method name (which we can grab from folder names)
METHODS = [
    ("Nerfacto Pseudo GT   ", "nerfacto---visibility-split"),
    ("Nerfacto             ", "nerfacto"),
    (" + Visibility Loss   ", "nerfacto-frustum"),
    (" + Vis + Sparsity    ", "nerfacto-frustum-sparsity"),
    (" + Vis + TV          ", "nerfacto-frustum-TV"),
    (" + Vis + RegNeRF     ", "nerfacto-frustum-regnerf"),
    (" + Vis + DSDS (Ours) ", "nerfacto-frustum-cube"),
]
# uncomment this for ablations
# METHODS = [
#     ("Nerfacto GT", "nerfacto---visibility-split"),
#     ("Nerfacto          ", "nerfacto"),
#     (" + Visibility Loss", "nerfacto-frustum"),
#     (" + Vis + RegNeRF  ", "nerfacto-frustum-regnerf"),
#     (" + Vis + Sparsity ", "nerfacto-frustum-sparsity"),
#     (" + Vis + TV       ", "nerfacto-frustum-TV"),
#     (" + Vis + Cube     ", "nerfacto-frustum-cube"),
#     # sampling
#     (" + Vis + Cube (sampling-densities)    ", "nerfacto-frustum-cube-sampling-densities"),
#     (" + Vis + Cube (sampling-uniform)     ", "nerfacto-frustum-cube-sampling-uniform"),
#     # activation
#     (" + Vis + Cube (activation-sigmoid)     ", "nerfacto-frustum-cube-activation-sigmoid"),
#     (" + Vis + Cube (activation-clamp)     ", "nerfacto-frustum-cube-activation-clamp"),
#     # cubesize
#     (" + Vis + Cube (cubescale-10-20)     ", "nerfacto-frustum-cube-cubescale-10-20"),
#     (" + Vis + Cube (cubescale-01-20)    ", "nerfacto-frustum-cube-cubescale-01-20"),
# ]
DATASETS = [
    "aloe",
    "art",
    "car",
    "century",
    "flowers",
    "garbage",
    "picnic",
    "pikachu",
    "pipe",
    "plant",
    "roses",
    "table",
]
# uncomment this for ablations
# DATASETS = ['garbage']

def print_datasets(datasets, caption_title="caption title", TABLE_LABEL="label"):
    metrics = get_metrics(datasets, "nerfacto-frustum-sparsity", METRICS_LIST)
    metrics = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics, SIGFIGS)]

    # create the experiment lines
    EXPERIMENT_LINES = ["\n"]
    for method_name, experiment_name in METHODS:
        metrics = get_metrics(datasets, experiment_name, METRICS_LIST)
        metrics = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics, SIGFIGS)]
        line_string = f"{method_name} & {' & '.join(metrics)} \\\\ \n"
        EXPERIMENT_LINES.append(line_string)

    s = TEMPLATE.replace("COLUMNS", COLUMNS)
    s = s.replace("EXPERIMENT_LINES", " ".join(EXPERIMENT_LINES))
    s = s.replace("CAPTION_TITLE", caption_title)
    s = s.replace("TABLE_LABEL", TABLE_LABEL)
    print(s)


# for all the datasets individually
for i in range(len(DATASETS)):
    print_datasets(DATASETS[i : i + 1], caption_title=f"``{DATASETS[i]}\" capture quantitative evaluation", TABLE_LABEL=DATASETS[i])



\begin{table*}[]
    \centering
    \begin{tabular}{l|lll|ll|lllll|l} % the number of columns needed
        \toprule
        
 & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ & Depth $\downarrow$ & Disp. $\downarrow$ & Mean $^{\circ}$ $\downarrow$ & Median $^{\circ}$ $\downarrow$ & \% $30^{\circ}$ $\uparrow$ & Coverage $\uparrow$ \\
        \midrule
        
 Nerfacto Pseudo GT    & 24.02 & 0.8591 & 0.0801 & 0.00 & 0.00 & 0.01 & 0.00 & 1.00 & 0.84 \\ 
 Nerfacto              & 18.33 & 0.6037 & 0.2968 & 0.01 & 0.04 & 45.50 & 39.01 & 0.37 & 0.84 \\ 
  + Visibility Loss    & 18.32 & 0.6063 & 0.2929 & 0.01 & 0.02 & 45.28 & 38.81 & 0.37 & 0.84 \\ 
  + Vis + Sparsity     & 18.41 & 0.6082 & 0.2895 & 0.01 & 0.02 & 45.19 & 38.76 & 0.37 & 0.84 \\ 
  + Vis + TV           & 18.31 & 0.6073 & 0.2983 & 0.01 & 0.02 & 46.12 & 39.34 & 0.36 & 0.84 \\ 
  + Vis + RegNeRF      & 18.31 & 0.6081 & 0.2933 & 0.01 & 0.02 & 45.44 & 39.04 & 0.37 & 0.84 \\ 
  + Vis + DSDS (Ours)  & 18.75 & 0.6390 & 0.2441